<!-- Projeto desenvolvido por Rafael F. Santos - mfa.rafael@gmail.com-->
# <font color='blue'>Sistema de Recomendação de Ações da B3 utilizando Vetores e Espaço Vetorial</font>

### 1. Definição do Problema:

**Objetivo:**  
Desenvolver um sistema de recomendação de ações da B3 (Bolsa de Valores do Brasil) que, dado um ticker informado, sugira outras ações relacionadas com base na classificação setorial das empresas. A solução deve considerar os seguintes critérios de classificação:  

1. **Setor Econômico**  
2. **Subsetor**  
3. **Segmento**  
4. **Segmento de Listagem da B3**

**Motivação:**  
Investidores frequentemente buscam diversificar suas carteiras com ações de empresas que possuam semelhanças setoriais ou que operem em mercados relacionados. Entretanto, a identificação manual dessas ações pode ser demorada e suscetível a erros. Um sistema de recomendação automatizado pode auxiliar nesse processo, oferecendo sugestões precisas e fundamentadas para facilitar a tomada de decisão.

**Desafio:**  
- Criar um modelo capaz de analisar o ticker fornecido e, com base na classificação setorial, identificar ações correlacionadas.  
- Garantir que o sistema seja capaz de lidar com atualizações periódicas nos dados das empresas listadas na B3.  
- Gerar recomendações úteis e relevantes para diferentes perfis de investidores.

**Produto Esperado:**  
Um sistema de recomendação que forneça uma lista de ações relacionadas ao ticker informado, permitindo aos usuários explorar outras opções dentro do mesmo setor ou setores correlacionados, com transparência nos critérios de recomendação.

### 2. Instalando e Carregando Pacotes

Os pacotes a seguir, são requisitos para o projeto e devem ser instalados ou atualizados.

In [1]:
# -*- coding: utf-8 -*-

# Instalação/atualização de pacotes, se necessário!

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.
!pip install -q -U nltk
!pip install -q -U numpy
!pip install -q -U pandas
!pip install -q -U scikit-learn
!pip install -q -U watermark
!pip install -q -U openpyxl

In [2]:
# Importação das bibliotecas
import ast
import nltk
import numpy as np
import pandas as pd
import sklearn
import warnings
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Rafael F. Santos"

Author: Rafael F. Santos



### 3. Coleta de Dados

A **classificação setorial da B3** organiza as empresas listadas na bolsa brasileira em categorias hierárquicas, facilitando a análise e comparação entre companhias de setores semelhantes. Essa estrutura é dividida em três níveis:

1. **Setor Econômico**: a categoria mais ampla, agrupando empresas de acordo com a natureza geral de suas atividades.

2. **Subsetor**: uma subdivisão dentro do setor econômico, que refina a classificação com base em características mais específicas das operações das empresas.

3. **Segmento**: a categoria mais específica, detalhando ainda mais as particularidades das atividades empresariais.

Essa classificação auxilia investidores e analistas a compreenderem melhor o mercado, permitindo comparações mais precisas entre empresas de um mesmo segmento ou setor.

Para acessar a classificação setorial completa e atualizada, a B3 disponibiliza um documento oficial em seu site. Você pode baixá-lo diretamente através do seguinte link:

[Classificação Setorial - B3](https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/acoes/consultas/classificacao-setorial/)

Este documento é uma referência valiosa para quem deseja aprofundar-se na estrutura setorial das empresas negociadas na B3 e é a base de dados que será utilizada neste projeto.

Além da **classificação setorial da B3**, também levaremos em consideração os **segmentos de de listagem da B3**.

Os segmentos de listagem definem o nível de **governança corporativa** das empresas que participam da B3. As empresas que aderem a um desses segmentos seguem regras de governança corporativa diferenciadas que vão além das obrigações legais brasileiras.

Também será considerado neste projeto, os BDRs (*Brazilian Depositary Receipts*), certificados de ações emitidas por empresas em outros países, mas negociadas normalmente na bolsa brasileira.

Deste modo, as categorias consideradas são:

* (DR1) BDR Nível 1
* (DR2) BDR Nível 2
* (DR3) BDR Nível 3
* (N1) Nível 1 de Governança Corporativa
* (N2) Nível 2 de Governança Corporativa
* (NM) Novo Mercado
* (MA) Bovespa Mais
* (M2) Bovespa Mais - Nível 2
* (MB) Balcão Organizado Tradicional

Para as empresas não listadas em uma das categorias acima, adotamos **Bolsa** como listagem padrão.


In [4]:
# Carregando e compreendendo os dados
df_original = pd.read_excel('dados/setorial.xlsx')

In [5]:
# Verificando a dimensão do dataset
df_original.shape

(538, 5)

In [6]:
# Amostragem dos dados
df_original.head(10)

,CLASSIFICAÇÃO SETORIAL DAS EMPRESAS NEGOCIADAS NA B3,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,LISTAGEM,NaN
6,NaN,NaN,NaN,CÓDIGO,SEGMENTO
7,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN,NaN
8,NaN,NaN,BRAVA,BRAV,NM
9,NaN,NaN,COSAN,CSAN,NM


In [7]:
# Amostragem dos dados
df_original.tail(10)

,CLASSIFICAÇÃO SETORIAL DAS EMPRESAS NEGOCIADAS NA B3,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
528,(M2) Bovespa Mais - Nível 2,NaN,NaN,NaN,NaN
529,(MB) Balcão Organizado Tradicional,NaN,NaN,NaN,NaN
530,NaN,NaN,NaN,NaN,NaN
531,ATENÇÃO,NaN,NaN,NaN,NaN
532,Este trabalho não é uma recomendação de invest...,NaN,NaN,NaN,NaN
533,As informações recebidas das empresas admitida...,NaN,NaN,NaN,NaN
534,nosso site www.b3.com.br.,NaN,NaN,NaN,NaN
535,"Para mais esclarecimentos, sugerimos procurar ...",NaN,NaN,NaN,NaN
536,potenciais das negociações com valores mobiliá...,NaN,NaN,NaN,NaN
537,"B3 S.A. - Brasil, Bolsa, Balcão",NaN,NaN,NaN,NaN


### 4. Tratamento e Análise Exploratória dos dados.

Neste passo, realizaremos o tratamento do dataset para prepará-lo para a aplicação do modelo de Machine Learning. As principais ações incluem:

* Renomeação de colunas para tornar os nomes mais intuitivos.
* Criação da variável 'empresa' e reorganização das colunas para melhorar a legibilidade.
* Exclusão de linhas desnecessárias e tratamento de dados nulos.
* Padronização e formatação dos dados para garantir consistência.
  
Essas etapas visam padronizar o dataset, aprimorar sua legibilidade e adequá-lo ao modelo de Machine Learning.

In [8]:
# Criando um dataset para os ajustes
df_ajustado = df_original

In [9]:
# Renomeando as colunas
df_ajustado.rename(columns={'CLASSIFICAÇÃO SETORIAL DAS EMPRESAS NEGOCIADAS NA B3': 'setor_economico', \
                    'Unnamed: 1': 'subsetor', 'Unnamed: 2': 'segmento', 'Unnamed: 3': 'codigo', \
                    'Unnamed: 4': 'listagem'}, inplace=True)

In [10]:
# Criando a coluna 'empresa'
df_ajustado['empresa'] = df_ajustado['segmento']

In [11]:
# Reorganizando as colunas
nova_ordem = ['empresa', 'codigo', 'setor_economico', 'subsetor', 'segmento', 'listagem' ]
df_ajustado = df_ajustado[nova_ordem]

In [12]:
# Visualizando o dataset
df_ajustado.head(10)

,empresa,codigo,setor_economico,subsetor,segmento,listagem
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,SEGMENTO,LISTAGEM,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,NaN
6,NaN,CÓDIGO,NaN,NaN,NaN,SEGMENTO
7,"Exploração, Refino e Distribuição",NaN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN
8,BRAVA,BRAV,NaN,NaN,BRAVA,NM
9,COSAN,CSAN,NaN,NaN,COSAN,NM


In [13]:
# Excluíndo linhas e resetando o índice
df_ajustado = df_ajustado.drop(index=range(7)).reset_index(drop=True)

In [14]:
# Preenchendo valores ausentes nas colunas 'setor_economico' e 'subsetor' com valores válidos
#  anteriores (forward fill).

# Para a coluna 'setor_economico':
# - O método `where` é usado para preservar os valores atuais da coluna onde a condição é verdadeira.
# - A condição verifica se o valor na coluna 'codigo' não é nulo (notna()) OU se o valor na própria
# coluna 'setor_economico' não é nulo.
# - Caso a condição seja falsa, o valor será preenchido com o último valor válido anterior,
# utilizando o método `ffill` (forward fill).
df_ajustado['setor_economico'] = df_ajustado['setor_economico'].where(df_ajustado['codigo'].notna() | \
                                                                      df_ajustado['setor_economico'].notna()).ffill()

# Para a coluna 'subsetor':
# - A lógica é a mesma aplicada à coluna 'setor_economico'.
# - Os valores são preservados onde 'codigo' não é nulo OU 'subsetor' não é nulo.
# - Caso contrário, os valores ausentes são preenchidos com o último valor válido anterior,
# utilizando o método `ffill`.
df_ajustado['subsetor'] = df_ajustado['subsetor'].where(df_ajustado['codigo'].notna() | \
                                                        df_ajustado['subsetor'].notna()).ffill()


In [15]:
# Preenchendo valores ausentes na coluna 'segmento' com o último valor válido anterior (forward fill),
# mas apenas para linhas onde a coluna 'codigo' é nula (isna()).

# Explicação do funcionamento:
# - O método `where` preserva os valores atuais da coluna 'segmento' apenas nas linhas onde a
# condição é verdadeira.
# - A condição verifica se o valor na coluna 'codigo' é nulo (isna()).
# - Para as linhas onde a condição é falsa (ou seja, onde 'codigo' NÃO é nulo), os valores ausentes
# são preenchidos com o último valor válido anterior, utilizando o método `ffill` (forward fill).
df_ajustado['segmento'] = df_ajustado['segmento'].where(df_ajustado['codigo'].isna()).ffill()

In [16]:
# Visualizando o dataset
df_ajustado.head(10)

,empresa,codigo,setor_economico,subsetor,segmento,listagem
0,"Exploração, Refino e Distribuição",NaN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN
1,BRAVA,BRAV,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
2,COSAN,CSAN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
3,PET MANGUINH,RPMG,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NaN
4,PETROBRAS,PETR,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2
5,PETRORECSA,RECV,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
6,PETRORIO,PRIO,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
7,RAIZEN,RAIZ,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2
8,ULTRAPAR,UGPA,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
9,VIBRA,VBBR,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM


In [17]:
# Preenchendo valores ausentes na coluna 'listagem' com o valor padrão 'Bolsa'.
df_ajustado['listagem'] = df_ajustado['listagem'].fillna('Bolsa')

In [18]:
# Removendo todas as linhas do dataset onde a coluna 'codigo' apresenta valores nulos
df_ajustado.dropna(subset=['codigo'], inplace=True)

In [19]:
# Visualizando o dataset
df_ajustado.head(10)

,empresa,codigo,setor_economico,subsetor,segmento,listagem
1,BRAVA,BRAV,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
2,COSAN,CSAN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
3,PET MANGUINH,RPMG,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",Bolsa
4,PETROBRAS,PETR,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2
5,PETRORECSA,RECV,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
6,PETRORIO,PRIO,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
7,RAIZEN,RAIZ,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2
8,ULTRAPAR,UGPA,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
9,VIBRA,VBBR,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
11,LUPATECH,LUPA,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",Equipamentos e Serviços,NM


In [20]:
# Contagem da frequência de cada valor único na coluna 'codigo'
df_ajustado['codigo'].value_counts()

codigo
CÓDIGO      12
LISTAGEM    12
OIBR         1
BRIT         1
LVTC         1
            ..
JOPA         1
CAML         1
MNPR         1
BEEF         1
GMAT         1
Name: count, Length: 394, dtype: int64

In [21]:
# Excluíndo os valores listados como 'CÓDIGO' E 'LISTAGEM' da coluna 'codigo'
df_ajustado.drop(df_ajustado.query('codigo == "CÓDIGO" | codigo == "LISTAGEM"').index, axis=0, inplace=True)

In [22]:
# Resetando o indice do dataset
df_ajustado.reset_index(drop=True, inplace=True)

In [23]:
# Amostragem do dataset
df_ajustado.head(10)

,empresa,codigo,setor_economico,subsetor,segmento,listagem
0,BRAVA,BRAV,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
1,COSAN,CSAN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
2,PET MANGUINH,RPMG,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",Bolsa
3,PETROBRAS,PETR,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2
4,PETRORECSA,RECV,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
5,PETRORIO,PRIO,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
6,RAIZEN,RAIZ,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2
7,ULTRAPAR,UGPA,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
8,VIBRA,VBBR,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
9,LUPATECH,LUPA,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis",Equipamentos e Serviços,NM


In [24]:
# Amostragem do dataset
df_ajustado.tail(10)

,empresa,codigo,setor_economico,subsetor,segmento,listagem
382,BETAPART,BETP,Outros,Outros,Outros,Bolsa
383,CEMEPE,MAPT,Outros,Outros,Outros,Bolsa
384,CIMS,CMSA,Outros,Outros,Outros,Bolsa
385,GAMA PART,OPGM,Outros,Outros,Outros,Bolsa
386,INVEST BEMGE,FIGE,Outros,Outros,Outros,Bolsa
387,POLPAR,PPAR,Outros,Outros,Outros,Bolsa
388,PROMPT PART,PRPT,Outros,Outros,Outros,Bolsa
389,SUDESTE S/A,OPSE,Outros,Outros,Outros,Bolsa
390,SUL 116 PART,OPTS,Outros,Outros,Outros,Bolsa
391,YBYRA S/A,YBRA,Outros,Outros,Outros,Bolsa


In [25]:
# Verificando a dimensão do dataset ajustado
df_ajustado.shape

(392, 6)

### 5. Modelagem de dados

Vamos modelar os dados do dataset para permitir o **processamento de texto** com ***Abstract Syntax Trees (ast)*** .

O módulo **ast** permite que aplicações Python processse árvores de gramática de sintaxe abstrata, o que ajuda a descobrir programaticamente como é a gramática atual.

In [26]:
# Separamos as strings das colunas setor_economico, subsetor, segmento e listagem
df_ajustado['setor_economico'] = df_ajustado['setor_economico'].apply(lambda x: x.split())
df_ajustado['subsetor'] = df_ajustado['subsetor'].apply(lambda x: x.split())
df_ajustado['segmento'] = df_ajustado['segmento'].apply(lambda x: x.split())
df_ajustado['listagem'] = df_ajustado['listagem'].apply(lambda x: x.split())

In [27]:
# Replace de espaço por vazio (remove espaços)
df_ajustado['setor_economico'] = df_ajustado['setor_economico'].apply(lambda x:[i.replace(" ","") for i in x])
df_ajustado['subsetor'] = df_ajustado['subsetor'].apply(lambda x:[i.replace(" ","") for i in x])
df_ajustado['segmento'] = df_ajustado['segmento'].apply(lambda x:[i.replace(" ","") for i in x])
df_ajustado['listagem'] = df_ajustado['listagem'].apply(lambda x:[i.replace(" ","") for i in x])


In [28]:
# Visualização do dataset
df_ajustado.head(20)

,empresa,codigo,setor_economico,subsetor,segmento,listagem
0,BRAVA,BRAV,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM]
1,COSAN,CSAN,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM]
2,PET MANGUINH,RPMG,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[Bolsa]
3,PETROBRAS,PETR,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[N2]
4,PETRORECSA,RECV,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM]
5,PETRORIO,PRIO,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM]
6,RAIZEN,RAIZ,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[N2]
7,ULTRAPAR,UGPA,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM]
8,VIBRA,VBBR,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM]
9,LUPATECH,LUPA,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Equipamentos, e, Serviços]",[NM]


In [29]:
# Criando a coluna tags, nesse caso um vetor de strings com os valores das colunas
df_ajustado['tags'] = df_ajustado['setor_economico'] + \
                        df_ajustado['subsetor'] + \
                        df_ajustado['segmento'] + \
                        df_ajustado['listagem']                      
                    

In [30]:
df_ajustado.head(10)

,empresa,codigo,setor_economico,subsetor,segmento,listagem,tags
0,BRAVA,BRAV,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
1,COSAN,CSAN,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
2,PET MANGUINH,RPMG,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[Bolsa],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
3,PETROBRAS,PETR,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[N2],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
4,PETRORECSA,RECV,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
5,PETRORIO,PRIO,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
6,RAIZEN,RAIZ,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[N2],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
7,ULTRAPAR,UGPA,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
8,VIBRA,VBBR,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Exploração,, Refino, e, Distribuição]",[NM],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
9,LUPATECH,LUPA,"[Petróleo,, Gás, e, Biocombustíveis]","[Petróleo,, Gás, e, Biocombustíveis]","[Equipamentos, e, Serviços]",[NM],"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."


In [31]:
# Dataset final
df_ajustado = df_ajustado[['empresa', 'codigo', 'tags']]

In [32]:
# Visualizando o dataset final
df_ajustado.head(10)

,empresa,codigo,tags
0,BRAVA,BRAV,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
1,COSAN,CSAN,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
2,PET MANGUINH,RPMG,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
3,PETROBRAS,PETR,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
4,PETRORECSA,RECV,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
5,PETRORIO,PRIO,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
6,RAIZEN,RAIZ,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
7,ULTRAPAR,UGPA,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
8,VIBRA,VBBR,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."
9,LUPATECH,LUPA,"[Petróleo,, Gás, e, Biocombustíveis, Petróleo,..."


In [33]:
# Join das strings paraq simplificar o vetor
df_ajustado['tags'] = df_ajustado['tags'].apply(lambda x: ' '.join(x))

In [34]:
# Colocando tudo em minúsculo para evitar diferença de palavras maiúsculo/minúsculo
df_ajustado['tags'] = df_ajustado['tags'].apply(lambda x: x.lower())

In [35]:
# Dataset final
df_final = df_ajustado

In [36]:
# Visualizandos o dataset final
df_final.head(10)

,empresa,codigo,tags
0,BRAVA,BRAV,"petróleo, gás e biocombustíveis petróleo, gás ..."
1,COSAN,CSAN,"petróleo, gás e biocombustíveis petróleo, gás ..."
2,PET MANGUINH,RPMG,"petróleo, gás e biocombustíveis petróleo, gás ..."
3,PETROBRAS,PETR,"petróleo, gás e biocombustíveis petróleo, gás ..."
4,PETRORECSA,RECV,"petróleo, gás e biocombustíveis petróleo, gás ..."
5,PETRORIO,PRIO,"petróleo, gás e biocombustíveis petróleo, gás ..."
6,RAIZEN,RAIZ,"petróleo, gás e biocombustíveis petróleo, gás ..."
7,ULTRAPAR,UGPA,"petróleo, gás e biocombustíveis petróleo, gás ..."
8,VIBRA,VBBR,"petróleo, gás e biocombustíveis petróleo, gás ..."
9,LUPATECH,LUPA,"petróleo, gás e biocombustíveis petróleo, gás ..."


## Parse e Vetorização

Stemming é o processo de redução de uma palavra ao seu radical que está ligado a sufixos e prefixos ou às raízes de palavras conhecidas como "lemmas". Stemming é importante na compreensão de linguagem natural e processamento de linguagem natural.

In [37]:
# Criando o parser
parser_ps = PorterStemmer()

In [38]:
# Função de stemming
def stem(text):
    
    # Cria uma lista vazia chamada y para armazenar as palavras após o stemming.
    y = []
    
    # Divide a string de entrada 'text' em palavras e itera sobre elas.
    for i in text.split():
        
        # Realiza o stemming na palavra atual 'i' e adiciona o resultado à lista y.
        y.append(parser_ps.stem(i))
    
    # Retorna as palavras processadas como uma string, unindo-as com espaços.
    return " ".join(y)

In [39]:
# Aplica a Função stem à coluna tags
df_final['tags'] = df_final['tags'].apply(stem)

In [40]:
# Visualizando o dataset
df_final.head(10)

,empresa,codigo,tags
0,BRAVA,BRAV,"petróleo, gá e biocombustívei petróleo, gá e b..."
1,COSAN,CSAN,"petróleo, gá e biocombustívei petróleo, gá e b..."
2,PET MANGUINH,RPMG,"petróleo, gá e biocombustívei petróleo, gá e b..."
3,PETROBRAS,PETR,"petróleo, gá e biocombustívei petróleo, gá e b..."
4,PETRORECSA,RECV,"petróleo, gá e biocombustívei petróleo, gá e b..."
5,PETRORIO,PRIO,"petróleo, gá e biocombustívei petróleo, gá e b..."
6,RAIZEN,RAIZ,"petróleo, gá e biocombustívei petróleo, gá e b..."
7,ULTRAPAR,UGPA,"petróleo, gá e biocombustívei petróleo, gá e b..."
8,VIBRA,VBBR,"petróleo, gá e biocombustívei petróleo, gá e b..."
9,LUPATECH,LUPA,"petróleo, gá e biocombustívei petróleo, gá e b..."


In [41]:
# Cria o vetorizador com no máximo 5000 atributos
cv = CountVectorizer(encoding='utf-8', max_features=500)

**CountVectorizer** é uma técnica de vetorização que converte uma coleção de documentos de texto em uma matriz de contagem de tokens contando quantas vezes cada palavra ocorre em cada documento.

No contexto de **processamento de linguagem natural (PLN)**, a vetorização é o processo de converter texto em uma representação númerica na forma de vetores. Este processo é fundamental para que algoritmos de ***Machine Learning** consigam trabalhar com dados textuais, visto que requerem entradas numérias.

Usamos dois parâmetros para a aplicação da técnica. O `encoding`, para identificar o padrão de codificação de caracteres e o `max_features`, para limitar o número de palavras mais frequentes a serem consideradas. Neste caso, limitado as 500 palavras mais frequentes.

In [42]:
# Cria os vetores para as tags
vectors = cv.fit_transform(df_final['tags']).toarray()

In [43]:
# Quantidade de palavras identificadas
len(cv.get_feature_names_out())

167

In [44]:
# Identificando o tipo da variável vectors
type(vectors)

numpy.ndarray

In [45]:
# Visualizando a variável vectors
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(392, 167))

In [46]:
# Para visualizar todas as colunas do array
np.set_printoptions(threshold = np.inf)

A matriz **vectors** gerada pela função fit_transform do CountVectorizer representa a frequência de cada palavra (após a aplicação de stemming) em cada documento (ou linha) da coluna de tags de seu dataframe. Aqui está o que significa cada elemento dessa matriz:

0: Indica que a palavra correspondente a essa posição na matriz não aparece na linha específica do dataframe. Ou seja, a tag após o stemming não está presente naquele registro específico.

1: Significa que a palavra aparece uma vez na linha do dataframe. Após o stemming, a tag foi identificada uma vez naquele registro.

2 (ou mais): Indica que a palavra aparece múltiplas vezes. Um valor de 2 significa que, após o stemming, a tag específica foi encontrada duas vezes naquele registro, e assim por diante para valores maiores.

Este comportamento é esperado porque o CountVectorizer constrói um vocabulário de todas as palavras únicas encontradas nos documentos fornecidos e, em seguida, conta quantas vezes cada palavra aparece em cada documento. O processo de stemming aplicado anteriormente por meio da função stem reduz as palavras a suas raízes, o que pode resultar na agregação de diferentes formas da mesma palavra como uma única entrada no vocabulário. Isso pode levar a um aumento na contagem (por exemplo, "run", "running", "runs" após stemming podem ser contados como a mesma palavra base, aumentando sua contagem em um documento específico).

In [47]:
vectors[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [48]:
vectors[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### 6. Criação do Modelo de Machine Learning

Utilizamos o método de distância do cosseno(consige_similarity) para calcular as distâncias matemáticas entre os vetores no espaço de dimensão.

In [49]:
# Calcula a similaridade entre os vetores calculando as distâncias entre eles
similaridades = cosine_similarity(vectors)

Por fim, construímos a função do sistema de recomendação.

In [66]:
# Função para o sistema de recomendação
def recomendAcao(codigo):
    
    # Obtém o índice da empresa passado como argumento o codigo 
    index = df_final[df_final['codigo'] == codigo].index[0]
    
    # Verificamos então as empresas com vetores de menor distância para o código passado como argumento
    distances = sorted(list(enumerate(similaridades[index])), reverse = True, key = lambda x: x[1])
    
    # E então consideramos as 5 empresas com menor distância, ou seja, maior similaridade
    for i in distances[1:6]:
        print(df_final.iloc[i[0]]['empresa'])

### 7. Avaliação e aplicação do Sistema de Recomendação

Criado o modelo, vamos avaliar os resultados gerados.

In [67]:
# Quais as recomendações de Empresas, com base na Classificação Setorial da B3, 
# para quem tem o ticker: 'AZUL'?
recomendAcao('AZUL')

GOL         
MARCOPOLO   
CCR SA      
CONC RIO TER
ECORODOVIAS 


In [68]:
# Quais as recomendações de Empresas, com base na Classificação Setorial da B3, 
# para quem tem o ticker: 'ABEV'?
recomendAcao('ABEV')

ALIPERTI    
GRANJA FARIA
EXCELSIOR   
MINUPAR     
JOSAPAR     


In [71]:
# Quais as recomendações de Empresas, com base na Classificação Setorial da B3, 
# para quem tem o ticker: 'AZUL'?
recomendAcao('BALM')

LIFEMED
INTELBRAS
MULTILASER
POSITIVO TEC
SCHULZ      


### Conclusão

O modelo desenvolvido apresentou um desempenho alinhado às expectativas, demonstrando sua eficácia em gerar recomendações com base na **Classificação Setorial da B3** e na **Listagem de Segmentos da B3**. Por meio do cálculo das distâncias vetoriais, utilizando o método de distância do cosseno (*cosine_similarity*), o sistema identifica empresas com alto grau de similaridade e correlação, em termos de características setoriais.

Essa solução é especialmente útil para usuários que buscam recomendações de empresas com perfis setoriais e de listagem semelhantes, apoiando de forma assertiva o processo de tomada de decisão. Ao fornecer indicações precisas, o modelo contribui para uma análise mais fundamentada no contexto do mercado brasileiro.

In [50]:
# Autoria do Projeto
%reload_ext watermark
%watermark -a "Rafael F. Santos"

Author: Rafael F. Santos



In [51]:
# Informações de sistema
%watermark -v -m

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.27.0

Compiler    : GCC 11.2.0
OS          : Linux
Release     : 6.6.6-76060606-generic
Machine     : x86_64
Processor   : 
CPU cores   : 20
Architecture: 64bit



In [52]:
# Versão dos pacotes utilizados
%watermark --iversions

sklearn: 1.6.0
numpy  : 2.2.1
nltk   : 3.9.1
pandas : 2.2.3

